In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
df = None
folder = 'hydrated_tweets/'
for filename in os.listdir(folder):
    if filename == '.DS_Store':
            continue
    newname = folder+filename
    for name in os.listdir(newname):
        if name == '.DS_Store':
            continue
        f = os.path.join(newname, name)
        if df is None:
            df = pd.read_csv(f, sep=';', engine='python',quotechar='"', error_bad_lines=False)
        else:
            tempdf = pd.read_csv(f, sep=';',engine='python',quotechar='"', error_bad_lines=False)
            df = pd.concat([df,tempdf], ignore_index=True)

print(df.keys())
print(df.head())


/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 6: ';' expected after '"'
Skipping line 20: ';' expected after '"'
Skipping line 24: ';' expected after '"'
Skipping line 28: ';' expected after '"'
Skipping line 30: ';' expected after '"'
Skipping line 31: ';' expected after '"'
Skipping line 32: ';' expected after '"'
Skipping line 33: ';' expected after '"'
Skipping line 34: ';' expected after '"'
Skipping line 38: ';' expected after '"'
Skipping line 40: ';' expected after '"'
Skipping line 42: ';' expected after '"'
Skipping line 43: ';' expected after '"'
Skipping line 45: ';' expected after '"'
Skipping line 48: ';' expected after '"'
Skipping line 49: ';' expected after '"'
Skipping line 51: ';' expected after '"'
Skipping line 55: ';' expected after '"'
Skipping line 57: ';' 

Skipping line 10218: ';' expected after '"'
Skipping line 10226: ';' expected after '"'
Skipping line 10227: ';' expected after '"'
Skipping line 10231: ';' expected after '"'
Skipping line 10233: ';' expected after '"'
Skipping line 10239: ';' expected after '"'
Skipping line 10243: ';' expected after '"'
Skipping line 10244: ';' expected after '"'
Skipping line 10246: ';' expected after '"'
Skipping line 10247: ';' expected after '"'
Skipping line 10251: ';' expected after '"'
Skipping line 10255: ';' expected after '"'
Skipping line 10257: ';' expected after '"'
Skipping line 10258: ';' expected after '"'
Skipping line 10259: ';' expected after '"'
Skipping line 10262: ';' expected after '"'
Skipping line 10263: ';' expected after '"'
Skipping line 10265: ';' expected after '"'
Skipping line 10270: ';' expected after '"'
Skipping line 10274: ';' expected after '"'
Skipping line 10276: ';' expected after '"'
Skipping line 10277: ';' expected after '"'
Skipping line 10283: ';' expecte

Skipping line 16218: ';' expected after '"'
Skipping line 16221: ';' expected after '"'
Skipping line 16223: ';' expected after '"'
Skipping line 16224: ';' expected after '"'
Skipping line 16226: ';' expected after '"'
Skipping line 16227: ';' expected after '"'
Skipping line 16228: ';' expected after '"'
Skipping line 16230: ';' expected after '"'
Skipping line 16231: ';' expected after '"'
Skipping line 16239: ';' expected after '"'
Skipping line 16241: ';' expected after '"'
Skipping line 16246: ';' expected after '"'
Skipping line 16248: ';' expected after '"'
Skipping line 16252: ';' expected after '"'
Skipping line 16253: ';' expected after '"'
Skipping line 16256: ';' expected after '"'
Skipping line 16261: ';' expected after '"'
Skipping line 16263: ';' expected after '"'
Skipping line 16265: ';' expected after '"'
Skipping line 16266: ';' expected after '"'
Skipping line 16268: ';' expected after '"'
Skipping line 16270: ';' expected after '"'
Skipping line 16274: ';' expecte

Skipping line 20679: ';' expected after '"'
Skipping line 20682: ';' expected after '"'
Skipping line 20683: ';' expected after '"'
Skipping line 20689: ';' expected after '"'
Skipping line 20695: ';' expected after '"'
Skipping line 20699: ';' expected after '"'
Skipping line 20702: ';' expected after '"'
Skipping line 20709: ';' expected after '"'
Skipping line 20710: ';' expected after '"'
Skipping line 20715: ';' expected after '"'
Skipping line 20717: ';' expected after '"'
Skipping line 20723: ';' expected after '"'
Skipping line 20724: ';' expected after '"'
Skipping line 20725: ';' expected after '"'
Skipping line 20733: ';' expected after '"'
Skipping line 20735: ';' expected after '"'
Skipping line 20740: ';' expected after '"'
Skipping line 20744: ';' expected after '"'
Skipping line 20747: ';' expected after '"'
Skipping line 20754: ';' expected after '"'
Skipping line 20758: ';' expected after '"'
Skipping line 20760: ';' expected after '"'
Skipping line 20763: ';' expecte

Skipping line 27690: ';' expected after '"'
Skipping line 27692: ';' expected after '"'
Skipping line 27694: ';' expected after '"'
Skipping line 27699: ';' expected after '"'
Skipping line 27700: ';' expected after '"'
Skipping line 27702: ';' expected after '"'
Skipping line 27703: ';' expected after '"'
Skipping line 27707: ';' expected after '"'
Skipping line 27708: ';' expected after '"'
Skipping line 27709: ';' expected after '"'
Skipping line 27710: ';' expected after '"'
Skipping line 27714: ';' expected after '"'
Skipping line 27720: ';' expected after '"'
Skipping line 27727: ';' expected after '"'
Skipping line 27728: ';' expected after '"'
Skipping line 27729: ';' expected after '"'
Skipping line 27733: ';' expected after '"'
Skipping line 27736: ';' expected after '"'
Skipping line 27740: ';' expected after '"'
Skipping line 27747: ';' expected after '"'
Skipping line 27748: ';' expected after '"'
Skipping line 27750: ';' expected after '"'
Skipping line 27751: ';' expecte

...

Skipping line 88001: ';' expected after '"'
Skipping line 88002: ';' expected after '"'
Skipping line 88003: ';' expected after '"'
Skipping line 88004: ';' expected after '"'
Skipping line 88010: ';' expected after '"'
Skipping line 88012: ';' expected after '"'
Skipping line 88016: ';' expected after '"'
Skipping line 88018: ';' expected after '"'
Skipping line 88019: ';' expected after '"'
Skipping line 88021: ';' expected after '"'
Skipping line 88023: ';' expected after '"'
Skipping line 88026: ';' expected after '"'
Skipping line 88027: ';' expected after '"'
Skipping line 88028: ';' expected after '"'
Skipping line 88030: ';' expected after '"'
Skipping line 88034: ';' expected after '"'
Skipping line 88036: ';' expected after '"'
Skipping line 88038: ';' expected after '"'
Skipping line 88040: ';' expected after '"'
Skipping line 88048: ';' expected after '"'
Skipping line 88058: ';' expected after '"'
Skipping line 88063: ';' expected after '"'
Skipping line 88065: ';' expecte

Skipping line 92342: ';' expected after '"'
Skipping line 92350: ';' expected after '"'
Skipping line 92353: ';' expected after '"'
Skipping line 92354: ';' expected after '"'
Skipping line 92355: ';' expected after '"'
Skipping line 92365: ';' expected after '"'
Skipping line 92367: ';' expected after '"'
Skipping line 92369: ';' expected after '"'
Skipping line 92371: ';' expected after '"'
Skipping line 92374: ';' expected after '"'
Skipping line 92377: ';' expected after '"'
Skipping line 92381: ';' expected after '"'
Skipping line 92384: ';' expected after '"'
Skipping line 92388: ';' expected after '"'
Skipping line 92391: ';' expected after '"'
Skipping line 92392: ';' expected after '"'
Skipping line 92395: ';' expected after '"'
Skipping line 92396: ';' expected after '"'
Skipping line 92403: ';' expected after '"'
Skipping line 92404: ';' expected after '"'
Skipping line 92405: ';' expected after '"'
Skipping line 92415: ';' expected after '"'
Skipping line 92419: ';' expecte

Skipping line 100406: ';' expected after '"'
Skipping line 100408: ';' expected after '"'
Skipping line 100410: ';' expected after '"'
Skipping line 100412: ';' expected after '"'
Skipping line 100414: ';' expected after '"'
Skipping line 100416: ';' expected after '"'
Skipping line 100423: ';' expected after '"'
Skipping line 100429: ';' expected after '"'
Skipping line 100431: ';' expected after '"'
Skipping line 100433: ';' expected after '"'
Skipping line 100436: ';' expected after '"'
Skipping line 100437: ';' expected after '"'
Skipping line 100442: ';' expected after '"'
Skipping line 100444: ';' expected after '"'
Skipping line 100446: ';' expected after '"'
Skipping line 100454: ';' expected after '"'
Skipping line 100457: ';' expected after '"'
Skipping line 100459: ';' expected after '"'
Skipping line 100461: ';' expected after '"'
Skipping line 100464: ';' expected after '"'
Skipping line 100465: ';' expected after '"'
Skipping line 100466: ';' expected after '"'
Skipping l

Skipping line 106911: ';' expected after '"'
Skipping line 106913: ';' expected after '"'
Skipping line 106914: ';' expected after '"'
Skipping line 106916: ';' expected after '"'
Skipping line 106918: ';' expected after '"'
Skipping line 106920: ';' expected after '"'
Skipping line 106930: ';' expected after '"'
Skipping line 106931: ';' expected after '"'
Skipping line 106933: ';' expected after '"'
Skipping line 106935: ';' expected after '"'
Skipping line 106940: ';' expected after '"'
Skipping line 106942: ';' expected after '"'
Skipping line 106944: ';' expected after '"'
Skipping line 106945: ';' expected after '"'
Skipping line 106947: ';' expected after '"'
Skipping line 106949: ';' expected after '"'
Skipping line 106951: ';' expected after '"'
Skipping line 106953: ';' expected after '"'
Skipping line 106963: ';' expected after '"'
Skipping line 106964: ';' expected after '"'
Skipping line 106968: ';' expected after '"'
Skipping line 106969: ';' expected after '"'
Skipping l

Index(['id', 'text', 'location', 'place', 'country', 'retweeted',
       'retweet_count', 'favorited', 'favorite_count', 'language', 'date',
       'verified', 'verified,,,,,,,,,,,,,,,,,,,,,,,,,,,,'],
      dtype='object')
                    id                                               text  \
0  1319111210864939011  RT @RepValDemings: Amy Coney Barrett ruled tha...   
1  1319111212194549761  RT @realDonaldTrump: Just don’t see any way Na...   
2  1319111212672667648  RT @ddale8: It turns out Trump has been making...   
3  1319111212743815169  RT @Laurie_Garrett: This would literally amoun...   
4  1319111212794249216  RT @JackPosobiec: I have never cursed on Twitt...   

           location place country retweeted retweet_count favorited  \
0            Kansas   NaN     NaN     False          3450     False   
1    Pittsburgh, PA   NaN     NaN     False         17984     False   
2     Ridgewood, NJ   NaN     NaN     False           462     False   
3     Virginia, USA   NaN     

In [17]:
df.to_csv(f'alltweets.csv',sep=';',index=False,header=True,encoding='utf-8')
#alltweetsdf = pd.read_csv('alltweets.csv',sep=';')
alltweetsdf = df
alltweetsdf.head()

,id,text,location,place,country,retweeted,retweet_count,favorited,favorite_count,language,date,verified,"verified,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
0,1319111210864939011,RT @RepValDemings: Amy Coney Barrett ruled tha...,Kansas,NaN,NaN,False,3450,False,0,en,10-22-03,False,NaN
1,1319111212194549761,RT @realDonaldTrump: Just don’t see any way Na...,"Pittsburgh, PA",NaN,NaN,False,17984,False,0,en,10-22-03,False,NaN
2,1319111212672667648,RT @ddale8: It turns out Trump has been making...,"Ridgewood, NJ",NaN,NaN,False,462,False,0,en,10-22-03,False,NaN
3,1319111212743815169,RT @Laurie_Garrett: This would literally amoun...,"Virginia, USA",NaN,NaN,False,1580,False,0,en,10-22-03,False,NaN
4,1319111212794249216,RT @JackPosobiec: I have never cursed on Twitt...,Fort Worth Texas,NaN,NaN,False,1857,False,0,en,10-22-03,False,NaN


In [6]:
print(alltweetsdf.shape)
print(alltweetsdf.isnull().sum(axis = 0))

(22305097, 13)
id                                             3
text                                       40927
location                                  100808
place                                   21995509
country                                 21992149
retweeted                                  41359
retweet_count                              41359
favorited                                  41359
favorite_count                             41359
language                                   41359
date                                       41363
verified                                  134637
verified,,,,,,,,,,,,,,,,,,,,,,,,,,,,    22215236
dtype: int64


In [7]:
# remove incorrectly parsed rows and tweets which aren't english
alltweetsdf = alltweetsdf.drop(alltweetsdf[(alltweetsdf['verified'].isnull()==True)].index)
alltweetsdf = alltweetsdf.drop(alltweetsdf[(alltweetsdf['text'].isnull()==True)].index)
alltweetsdf = alltweetsdf.drop(alltweetsdf[(alltweetsdf['language']!='en')].index)

In [8]:
# dropping certain columns
print((alltweetsdf['retweeted']==True).sum(axis = 0))
print((alltweetsdf['favorited']==True).sum(axis = 0))
alltweetsdf = alltweetsdf.drop(columns=['retweeted','favorited','language','verified,,,,,,,,,,,,,,,,,,,,,,,,,,,,'])

0
0


In [9]:
print(alltweetsdf.shape)
print(alltweetsdf.isnull().sum(axis = 0))
# bit more than half the tweets are retweets
print('Retweets:',alltweetsdf['text'].duplicated().sum())

(19303737, 9)
id                       0
text                     0
location             43067
place             19066901
country           19066921
retweet_count            0
favorite_count           0
date                     0
verified                 0
dtype: int64
Retweets: 12291948


In [10]:
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New York', 'New Mexico', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']
stateCodes = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
stateMapping = {'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NY': 'New York', 'NM': 'New Mexico', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT':  'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV':  'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'}

In [11]:
# some people have multiple states in their description, first state check is their given state
def findstate(x):
    location = x[0]
    place = x[1]
    #print('location',location,'place',place)
    for state in stateMapping:
        if f', {state}' in str(location) or stateMapping[state] in str(location) or f', {state}' in str(place) or stateMapping[state] in str(place):
            #print('state',state,'map',stateMapping[state],'location',location,'place',place)
            return stateMapping[state]
    return None
alltweetsdf["state"] = np.nan
alltweetsdf["state"] = alltweetsdf.apply(lambda row : findstate([row['location'],row['place']]), axis=1)

In [12]:
alltweetsdf.head(50)

,id,text,location,place,country,retweet_count,favorite_count,date,verified,state
0,1319111210864939011,RT @RepValDemings: Amy Coney Barrett ruled tha...,Kansas,NaN,NaN,3450,0,10-22-03,False,Kansas
1,1319111212194549761,RT @realDonaldTrump: Just don’t see any way Na...,"Pittsburgh, PA",NaN,NaN,17984,0,10-22-03,False,Pennsylvania
2,1319111212672667648,RT @ddale8: It turns out Trump has been making...,"Ridgewood, NJ",NaN,NaN,462,0,10-22-03,False,New Jersey
3,1319111212743815169,RT @Laurie_Garrett: This would literally amoun...,"Virginia, USA",NaN,NaN,1580,0,10-22-03,False,Virginia
4,1319111212794249216,RT @JackPosobiec: I have never cursed on Twitt...,Fort Worth Texas,NaN,NaN,1857,0,10-22-03,False,Texas
5,1319111212831887360,RT @Amy_Siskind: Note every bit of foreign int...,Colorado,NaN,NaN,694,0,10-22-03,False,Colorado
6,1319111212853022721,RT @donwinslow: Hunter Biden is not responsibl...,"Boston, MA",NaN,NaN,5215,0,10-22-03,False,Massachusetts
7,1319111213087924226,RT @willchamberlain: Maybe *you* never lived u...,One-Party-Rule California,NaN,NaN,602,0,10-22-03,False,California
8,1319111213222141952,RT @zachsilberberg: what a difficult week for ...,"Toronto, Ontario",NaN,NaN,586,0,10-22-03,False,None
9,1319111213238964226,RT @TCRG2012: @cynthia_ann @LisaWNC1 @Kelleyro...,Western Massachusetts,NaN,NaN,2,0,10-22-03,False,Massachusetts


In [13]:
print(alltweetsdf.shape)
print(alltweetsdf.isnull().sum(axis = 0))
print('Retweets:',alltweetsdf['text'].duplicated().sum())

(19303737, 10)
id                       0
text                     0
location             43067
place             19066901
country           19066921
retweet_count            0
favorite_count           0
date                     0
verified                 0
state              9855920
dtype: int64
Retweets: 12291948


In [14]:
# delete nan rows from states
# delete location place and country columns
# save to clean csv
alltweetsdf = alltweetsdf.drop(alltweetsdf[(alltweetsdf['state'].isnull()==True)].index)
alltweetsdf = alltweetsdf.drop(columns=['location','place','country'])

In [15]:
print(alltweetsdf.shape)
print(alltweetsdf.isnull().sum(axis = 0))
print('Retweets:',alltweetsdf['text'].duplicated().sum())

(9447817, 7)
id                0
text              0
retweet_count     0
favorite_count    0
date              0
verified          0
state             0
dtype: int64
Retweets: 5913702


In [16]:
cleantweets = alltweetsdf
cleantweets.to_csv(f'cleantweets.csv',sep=';',index=False,header=True,encoding='utf-8')